# LAB 7

Name: Akshay Kekuda

In [1106]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import re

from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.utils.extmath import randomized_svd
from nltk.tokenize import word_tokenize, wordpunct_tokenize

In [1107]:
source_df = pd.DataFrame()

# def remove_rt(string):
    
with open('Health-Tweets/nytimeshealth.txt') as f:
    for line in f:
        line = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", "", line)
        temp = line.split(sep= '|')
#         print(temp)
        string=''
        if len(temp) >2:
            for i in range(2, len(temp)):
                string = string + temp[i]
        string = re.sub(r"^.*?:", '', string)
        string = string.replace(' \n', '')
#         print(string)
        new_list=[temp[0], temp[1], string]
        to_append = pd.Series(new_list)
        source_df = source_df.append(to_append, ignore_index=True)
source_df                

,0,1,2
0,548662191340421120,Sat Dec 27 02:10:34 +0000 2014,Risks in Using Social Media to Spot Signs of M...
1,548579831169163265,Fri Dec 26 20:43:18 +0000 2014,The most effective nationwide diabetes preven...
2,548579045269852161,Fri Dec 26 20:40:11 +0000 2014,Diabetes Prevention That Works
3,548444679529041920,Fri Dec 26 11:46:15 +0000 2014,Comfort Casseroles for Winter Dinners
4,548311901227474944,Fri Dec 26 02:58:39 +0000 2014,High-Level Knowledge Before Veterans Affairs S...
...,...,...,...
6240,318864599728414720,Mon Apr 01 23:16:59 +0000 2013,Hpv Vaccines for Boys
6241,318860789899476992,Mon Apr 01 23:01:51 +0000 2013,Health Law Provision for Small Business Is Del...
6242,318860787605176321,Mon Apr 01 23:01:50 +0000 2013,Gay Men in Mexico City Would Stay H.I.V.-Free...
6243,318856950890766336,Mon Apr 01 22:46:35 +0000 2013,Multiple Sclerosis Balloon Therapy Fails in Study


In [1108]:
# source_df = pd.read_csv('Health-Tweets/nytimeshealth.txt', delimiter='|', header=None)

In [1109]:
source_df.head()

,0,1,2
0,548662191340421120,Sat Dec 27 02:10:34 +0000 2014,Risks in Using Social Media to Spot Signs of M...
1,548579831169163265,Fri Dec 26 20:43:18 +0000 2014,The most effective nationwide diabetes preven...
2,548579045269852161,Fri Dec 26 20:40:11 +0000 2014,Diabetes Prevention That Works
3,548444679529041920,Fri Dec 26 11:46:15 +0000 2014,Comfort Casseroles for Winter Dinners
4,548311901227474944,Fri Dec 26 02:58:39 +0000 2014,High-Level Knowledge Before Veterans Affairs S...


In [1110]:
source_df.columns = ['ID', 'TimeStamp', 'Tweet']

In [1111]:
source_df.head()

,ID,TimeStamp,Tweet
0,548662191340421120,Sat Dec 27 02:10:34 +0000 2014,Risks in Using Social Media to Spot Signs of M...
1,548579831169163265,Fri Dec 26 20:43:18 +0000 2014,The most effective nationwide diabetes preven...
2,548579045269852161,Fri Dec 26 20:40:11 +0000 2014,Diabetes Prevention That Works
3,548444679529041920,Fri Dec 26 11:46:15 +0000 2014,Comfort Casseroles for Winter Dinners
4,548311901227474944,Fri Dec 26 02:58:39 +0000 2014,High-Level Knowledge Before Veterans Affairs S...


In [1112]:
source_df.isna().sum()

ID           0
TimeStamp    0
Tweet        0
dtype: int64

In [1113]:
source_df.dtypes

ID           object
TimeStamp    object
Tweet        object
dtype: object

In [1114]:
source_df.ID=source_df.ID.astype(int)
source_df.dtypes

ID            int64
TimeStamp    object
Tweet        object
dtype: object

In [1115]:
source_df.TimeStamp=pd.to_datetime(source_df.TimeStamp)
source_df.head()

,ID,TimeStamp,Tweet
0,548662191340421120,2014-12-27 02:10:34+00:00,Risks in Using Social Media to Spot Signs of M...
1,548579831169163265,2014-12-26 20:43:18+00:00,The most effective nationwide diabetes preven...
2,548579045269852161,2014-12-26 20:40:11+00:00,Diabetes Prevention That Works
3,548444679529041920,2014-12-26 11:46:15+00:00,Comfort Casseroles for Winter Dinners
4,548311901227474944,2014-12-26 02:58:39+00:00,High-Level Knowledge Before Veterans Affairs S...


In [1116]:
source_df.dtypes

ID                         int64
TimeStamp    datetime64[ns, UTC]
Tweet                     object
dtype: object

In [1117]:
source_df.Tweet[0]

'Risks in Using Social Media to Spot Signs of Mental Distress'

In [1118]:
source_df.shape

(6245, 3)

In [1119]:
source_df.ID.is_unique

True

In [1120]:
source_df[source_df.Tweet.str.contains("#")].Tweet

42       Is farmed fish better than no fish at all?  #...
136      Stroke in #pregnancy is "rare, deadly and inc...
152      How NYT covers Ebola  Editor Greg Winter quot...
360      Should food containing pot require approval b...
420     A hearty salad for the non-meat eaters at Than...
                              ...                        
6065                                            #Ask Well
6068    New research suggest fathers can pick out thei...
6172     Remedies for Nail Fungus  Have a health quest...
6191    Have questions about the staying fit? Ask Gret...
6215    Straw-like tubes of modified gelatin provide p...
Name: Tweet, Length: 142, dtype: object

## Requirement 1

* I considered the NYTimes health data set for my analysis
* This text file has '|' as it's delimiters. The file has 3 attributes so we would expect two '|' but some of the tweets have '|' character in them. So pd.readcsv would not work directly. So I have deleted all the | from the Tweets. Occurence of | in the tweets was when there was subtype of heading like world briefing|health
* Tweets also have references to twitter users which is a string followed by @. They have not been deleted.
* All URLs have been deleted
* Converted the timestamp data column to pd timeseries data type
* There is no missing or duplicate data
* There are hashtags in the tweets. I have not deleted them 
* The NYTimes text file has 6245 tweets
* Many tweets contain the category of health care news at the start of tweet like 'RT @user_id:', 'Well:', 'The New Health Care:', 'The New Old Age Blog' etc. These dont give us the concept that we are looking at. So I have deleted them. Followed by the category, there are tweets which also have sub category. These have been preserved as they do contain useful information on the topic.
* 

## Requirement 2

In [1121]:
ps = PorterStemmer()

corpus_stemmed = []
for d in source_df.Tweet:
    words = pd.Series(wordpunct_tokenize(d))
    stemmed_words = words.apply(ps.stem)
    corpus_stemmed.append(' '.join(list(stemmed_words)))

print('We processed {} documents.'.format(len(corpus_stemmed)))

<ipython-input-1121-e109ad741456>:5: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  words = pd.Series(wordpunct_tokenize(d))


We processed 6245 documents.


In [1122]:
corpus_stemmed

['risk in use social media to spot sign of mental distress',
 "the most effect nationwid diabet prevent program you ' ve probabl never heard of :",
 'diabet prevent that work',
 'comfort casserol for winter dinner',
 'high - level knowledg befor veteran affair scandal',
 'afford care act ’ s tax effect now loom for filer',
 'christma in the hospit',
 'for member of health ministri in texa , care mean share the bill',
 'A q & amp ; a with laura hillenbrand , author of " unbroken ," who suffer from chronic fatigu syndrom .',
 'race to deliv nicotin ’ s punch , with less risk',
 'peopl are shop for health insur , surprisingli',
 'bedtim technolog for a better night ’ s sleep',
 'lab error may have expos U . S . technician to ebola viru',
 'C . D . C . ebola error in lab may have expos technician to viru',
 'break new : lab error may have expos U . S . technician to ebola viru',
 'A q & amp ; a with laura hillenbrand , author of " unbroken ," who suffer from chronic fatigu syndrom .',
 'vi

In [1123]:
tweet_label = [(lambda x: 'tweet{:d}'.format(x))(i) for i in range(source_df.shape[0])]
tweet_label

['tweet0',
 'tweet1',
 'tweet2',
 'tweet3',
 'tweet4',
 'tweet5',
 'tweet6',
 'tweet7',
 'tweet8',
 'tweet9',
 'tweet10',
 'tweet11',
 'tweet12',
 'tweet13',
 'tweet14',
 'tweet15',
 'tweet16',
 'tweet17',
 'tweet18',
 'tweet19',
 'tweet20',
 'tweet21',
 'tweet22',
 'tweet23',
 'tweet24',
 'tweet25',
 'tweet26',
 'tweet27',
 'tweet28',
 'tweet29',
 'tweet30',
 'tweet31',
 'tweet32',
 'tweet33',
 'tweet34',
 'tweet35',
 'tweet36',
 'tweet37',
 'tweet38',
 'tweet39',
 'tweet40',
 'tweet41',
 'tweet42',
 'tweet43',
 'tweet44',
 'tweet45',
 'tweet46',
 'tweet47',
 'tweet48',
 'tweet49',
 'tweet50',
 'tweet51',
 'tweet52',
 'tweet53',
 'tweet54',
 'tweet55',
 'tweet56',
 'tweet57',
 'tweet58',
 'tweet59',
 'tweet60',
 'tweet61',
 'tweet62',
 'tweet63',
 'tweet64',
 'tweet65',
 'tweet66',
 'tweet67',
 'tweet68',
 'tweet69',
 'tweet70',
 'tweet71',
 'tweet72',
 'tweet73',
 'tweet74',
 'tweet75',
 'tweet76',
 'tweet77',
 'tweet78',
 'tweet79',
 'tweet80',
 'tweet81',
 'tweet82',
 'tweet83',
 '

In [1124]:
stop_words = ["0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "affects", "after", "afterwards", "ag", "again", "against", "ah", "ain", "ain't", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anything", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appear", "appreciate", "appropriate", "approximately", "ar", "are", "aren", "arent", "aren't", "arise", "around", "as", "a's", "aside", "ask", "asking", "associated", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "begin", "beginning", "beginnings", "begins", "behind", "being", "believe", "below", "beside", "besides", "best", "better", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "c1", "c2", "c3", "ca", "call", "came", "can", "cannot", "cant", "can't", "cause", "causes", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "changes", "ci", "cit", "cj", "cl", "clearly", "cm", "c'mon", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "contain", "containing", "contains", "corresponding", "could", "couldn", "couldnt", "couldn't", "course", "cp", "cq", "cr", "cry", "cs", "c's", "ct", "cu", "currently", "cv", "cx", "cy", "cz", "d", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "didn't", "different", "dj", "dk", "dl", "do", "does", "doesn", "doesn't", "doing", "don", "done", "don't", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "effect", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "empty", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "first", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "h2", "h3", "had", "hadn", "hadn't", "happens", "hardly", "has", "hasn", "hasnt", "hasn't", "have", "haven", "haven't", "having", "he", "hed", "he'd", "he'll", "hello", "help", "hence", "her", "here", "hereafter", "hereby", "herein", "heres", "here's", "hereupon", "hers", "herself", "hes", "he's", "hh", "hi", "hid", "him", "himself", "his", "hither", "hj", "ho", "home", "hopefully", "how", "howbeit", "however", "how's", "hr", "hs", "http", "hu", "hundred", "hy", "i", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "i'd", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "i'll", "im", "i'm", "immediate", "immediately", "importance", "important", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "invention", "inward", "io", "ip", "iq", "ir", "is", "isn", "isn't", "it", "itd", "it'd", "it'll", "its", "it's", "itself", "iv", "i've", "ix", "iy", "iz", "j", "jj", "jr", "js", "jt", "ju", "just", "k", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "know", "known", "knows", "ko", "l", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "let's", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "mean", "means", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mightn't", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "mustn't", "my", "myself", "n", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "necessary", "need", "needn", "needn't", "needs", "neither", "never", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "no", "nobody", "non", "none", "nonetheless", "noone", "nor", "normally", "nos", "not", "noted", "nothing", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "other", "others", "otherwise", "ou", "ought", "our", "ours", "ourselves", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "possible", "possibly", "potentially", "pp", "pq", "pr", "predominantly", "present", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "s2", "sa", "said", "same", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "see", "seeing", "seem", "seemed", "seeming", "seems", "seen", "self", "selves", "sensible", "sent", "serious", "seriously", "seven", "several", "sf", "shall", "shan", "shan't", "she", "shed", "she'd", "she'll", "shes", "she's", "should", "shouldn", "shouldn't", "should've", "show", "showed", "shown", "showns", "shows", "si", "side", "significant", "significantly", "similar", "similarly", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somebody", "somehow", "someone", "somethan", "something", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "system", "sz", "t", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "that'll", "thats", "that's", "that've", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "there'll", "thereof", "therere", "theres", "there's", "thereto", "thereupon", "there've", "these", "they", "theyd", "they'd", "they'll", "theyre", "they're", "they've", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "t's", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "use", "used", "useful", "usefully", "usefulness", "uses", "using", "usually", "ut", "v", "va", "value", "various", "vd", "ve", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "wa", "want", "wants", "was", "wasn", "wasnt", "wasn't", "way", "we", "wed", "we'd", "welcome", "well", "we'll", "well-b", "went", "were", "we're", "weren", "werent", "weren't", "we've", "what", "whatever", "what'll", "whats", "what's", "when", "whence", "whenever", "when's", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "where's", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "who'll", "whom", "whomever", "whos", "who's", "whose", "why", "why's", "wi", "widely", "will", "willing", "wish", "with", "within", "without", "wo", "won", "wonder", "wont", "won't", "words", "world", "would", "wouldn", "wouldnt", "wouldn't", "www", "x", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "you'd", "you'll", "your", "youre", "you're", "yours", "yourself", "yourselves", "you've", "yr", "ys", "yt", "z", "zero", "zi", "zz"]

In [1125]:
medical_words=[ 'ask', 'well', 'studi', 'gretchenreynold', 'andrew', 'scrivani', 'nyt',
               #'hospit', 'medic','live', 'life', 'peopl',, 'health', 'drug','good'
               'thi', 'ha', 'wi', 'whi','letter']
stop_words = stop_words + medical_words

In [1126]:
vectorizer=CountVectorizer(stop_words=stop_words, token_pattern=r"(?u)\b\w[\w'’]*\w\b")

In [1127]:
X = np.asarray(vectorizer.fit_transform(corpus_stemmed).todense())
X.shape

/Users/akshaykekuda/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['articl'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


(6245, 5972)

## Requirement 3

The X matrix is of shape (6245, 5972)
We have 6245 tweets which correspond to the row count and 5972 terms in the vocabulary which correspond to the column count

In [1128]:
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [1129]:
vocab = vectorizer.get_feature_names()
len(vocab)

5972

In [1130]:
tf_df_raw = pd.DataFrame(
    X.T,
    index=vectorizer.get_feature_names(),
    columns = tweet_label)
tf_df_raw

,tweet0,tweet1,tweet2,tweet3,tweet4,tweet5,tweet6,tweet7,tweet8,tweet9,...,tweet6235,tweet6236,tweet6237,tweet6238,tweet6239,tweet6240,tweet6241,tweet6242,tweet6243,tweet6244
00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00am,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
05,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zombi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zone,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zoonot,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zucchini,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [1131]:
s = tf_df_raw.sum(axis = 1)
s.sort_values(ascending=False)[0:20]

ebola      660
health     454
cancer     279
doctor     269
patient    242
drug       227
care       211
risk       195
hospit     143
exercis    131
die        120
medic      115
life       114
test       113
breast     106
peopl      104
diseas     102
live       101
death      101
women      100
dtype: int64

## Requirement 4

The original top 20 words of the NYtimes data on the first run with the original stop word list is as follows: 


|Term|Count|
|---|---|
ebola|      660
health |    454
cancer  |   279
doctor |    269
patient|    242
drug  |     227
studi|      212
care |      211
risk |      195
hospit|     143
exercis|    131
thi |       125
die  |      120
ha    |     116
medic  |    115
life    |   114
test     |  113
whi       | 108
breast     |106
peopl      |104
    

As can be seen above we have some words like thi, ha and whi. These should be added to the stop word list. Adding the below terms to the stop_word lists helps us arrive at a better concept definition. 

Based on various trials of stop words, I arrived at the following additional stop words:

medical_words=[ 'ask', 'well', 'studi', 'gretchenreynold', 'andrew', 'scrivani', 'nyt',
               'thi', 'ha', 'wi', 'whi','letter']

During my analysis with stop words I found that "Ask Well", "Studies", and  'letters', were some sub categories that occured after the first category, which was deleted in the preprocessing part. Also the reporters 'gretchenreynold', 'andrew', 'scrivani' from 'nyt' occurred in the top 10 words in some concepts. So I have added these to the stop list

Based on the above additional stop words I have the following top 20 words:

|Term|Count|
|-:|:-|
ebola|      660
health|     454
cancer|     279
doctor|     269
patient|    242
drug|       227
care |      211
risk  |     195
hospit |    143
exercis |   131
die      |  120
medic     | 115
life|       114
test |      113
breast|     106
peopl  |    104
diseas  |   102
live     |  101
death     | 101
women      |100

In [1132]:
tfidf = TfidfTransformer()

In [1133]:
Y = np.asarray(tfidf.fit_transform(X).todense())
Y.shape

(6245, 5972)

## Requirement 5

The shape of the tfidf matrix is (6245, 5972) which is same as that of the count vectorizer

In [1134]:
# print idf values 
tf_idf_df = pd.DataFrame(Y.T, index=vectorizer.get_feature_names(), columns = tweet_label) 

In [1135]:
tf_idf_df

,tweet0,tweet1,tweet2,tweet3,tweet4,tweet5,tweet6,tweet7,tweet8,tweet9,...,tweet6235,tweet6236,tweet6237,tweet6238,tweet6239,tweet6240,tweet6241,tweet6242,tweet6243,tweet6244
00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00am,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zombi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zoonot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zucchini,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Requirement 6

In [1136]:
n_concepts = 10
concepts = [(lambda x: 'concept{:d}'.format(x))(i+1) for i in range(n_concepts)]
U, Sigma, VT = randomized_svd(Y, n_components=n_concepts)

In [1137]:
U.shape

(6245, 10)

In [1138]:
U_df = pd.DataFrame(
    U,
    index=tweet_label,
    columns=concepts)
U_df['content'] = source_df.Tweet.to_list()
U_df

,concept1,concept2,concept3,concept4,concept5,concept6,concept7,concept8,concept9,concept10,content
tweet0,0.005797,0.006545,0.002150,-0.002286,0.012250,0.022923,0.017124,-0.023721,-0.000252,-0.018275,Risks in Using Social Media to Spot Signs of M...
tweet1,0.003022,0.001688,0.000102,0.001059,0.008443,0.007931,0.003927,0.001373,0.005703,-0.002208,The most effective nationwide diabetes preven...
tweet2,0.006975,0.002555,0.000975,0.001385,0.014934,0.015709,0.009448,-0.000716,0.019766,-0.000194,Diabetes Prevention That Works
tweet3,0.000235,0.000139,0.000204,-0.000002,0.000458,0.000315,0.000314,-0.000515,-0.000304,0.000073,Comfort Casseroles for Winter Dinners
tweet4,0.003512,0.001508,0.002623,-0.000390,0.006473,0.001359,0.002673,-0.002000,0.003420,0.003191,High-Level Knowledge Before Veterans Affairs S...
...,...,...,...,...,...,...,...,...,...,...,...
tweet6240,0.004643,0.002740,-0.001743,-0.003025,0.002491,0.002961,0.001215,-0.003635,0.010926,0.021926,Hpv Vaccines for Boys
tweet6241,0.015551,0.009262,0.041452,-0.000987,-0.007632,-0.001975,-0.005564,-0.000422,-0.017628,-0.005815,Health Law Provision for Small Business Is Del...
tweet6242,0.004436,0.002898,-0.000595,-0.000579,0.002859,-0.000748,0.002859,0.000928,0.005320,0.004177,Gay Men in Mexico City Would Stay H.I.V.-Free...
tweet6243,0.000964,0.001428,-0.000570,0.000257,0.000426,-0.000974,0.002775,0.001868,-0.000378,0.002673,Multiple Sclerosis Balloon Therapy Fails in Study


In [1139]:
V_df = pd.DataFrame(
    VT.T,
    index=vocab,
    columns=concepts
)
V_df

,concept1,concept2,concept3,concept4,concept5,concept6,concept7,concept8,concept9,concept10
00,0.003499,0.001176,-0.002919,1.779852e-02,-0.002308,0.000502,-0.010248,-0.007284,-0.000494,-0.002166
000,0.023497,-0.005585,-0.000170,-5.458537e-03,-0.000794,0.000125,-0.011519,-0.013016,0.011321,0.010726
00am,0.001377,0.000374,-0.000981,6.701288e-03,-0.000707,0.000154,-0.003844,-0.002630,-0.000435,-0.000898
05,0.000086,0.000090,-0.000007,-2.743941e-05,0.000241,0.000415,0.000184,-0.000346,0.000018,0.000021
06,0.000829,0.000600,0.002505,1.469716e-05,-0.000401,0.000013,-0.000522,-0.000045,-0.001637,-0.000060
...,...,...,...,...,...,...,...,...,...,...
zombi,0.000700,0.000497,0.001959,1.332309e-04,0.000068,-0.000287,-0.000458,-0.000340,-0.000703,0.001578
zone,0.004630,-0.002502,-0.001546,-2.918381e-03,0.001333,0.003471,0.000012,-0.005371,0.000991,-0.004719
zoonot,0.000223,0.000109,0.000179,1.656193e-04,0.000246,0.000026,-0.000138,-0.000411,-0.000183,0.001328
zucchini,0.000018,0.000017,0.000079,-3.915608e-07,-0.000017,-0.000023,-0.000029,-0.000009,-0.000190,-0.000061


In [1140]:
top_term = pd.DataFrame(columns = concepts)
for col in concepts:
    top_term[col] = V_df[col].sort_values(ascending=False)[0:10].index

## Requirement 7

In [1141]:
top_term

,concept1,concept2,concept3,concept4,concept5,concept6,concept7,concept8,concept9,concept10
0,ebola,cancer,health,doctor,exercis,drug,patient,drug,care,heart
1,health,breast,care,patient,risk,risk,risk,patient,hospit,diseas
2,patient,live,law,solv,brain,heart,hospit,treatment,life,test
3,doctor,health,insur,medic,drug,diseas,heart,hospit,afford,medic
4,cancer,risk,exchang,eye,sleep,tie,care,approv,death,life
5,care,care,recip,care,eat,treatment,tie,experiment,cost,death
6,outbreak,women,plan,mysteri,heart,diabet,diseas,exercis,medic,rate
7,drug,drug,news,babi,weight,approv,older,treat,nurs,vaccin
8,hospit,stori,mental,lose,age,children,diabet,maker,long,good
9,risk,face,cost,shortag,good,increas,increas,gener,rise,pain


In [1142]:
top_tweet = pd.DataFrame(columns = concepts)
for col in concepts:
    top_tweet[col] = U_df.loc[U_df[col].sort_values(ascending=False)[0:10].index]['content'].values

## Requirement 8

In [1143]:
top_tweet

,concept1,concept2,concept3,concept4,concept5,concept6,concept7,concept8,concept9,concept10
0,"Studying Ebola, Then Dying From It",Where is Breast Cancer?,U.S. to End Coverage Under Health Care Law for...,Letters to the Doctor,Can You Get Too Much Exercise?,A.D.H.D. Treatment Without Drugs (1 Letter),Best Interests of the Patient (1 Letter),A.D.H.D. Treatment Without Drugs (1 Letter),Is the Affordable Care Act Working?,Heart Disease
1,What you need to know about the Ebola outbreak,"Look for Cancer, and Find It",What do you know about current health news? Te...,What Patients Don’t Tell Their Doctors,Ask Well: How Often to Exercise,One Drug or 2? Parents See Risk but Also Hope,"‘Good’ Patients, ‘Bad’ Patients",This Is Your Brain on Drugs,‘Accountable Care’ Helping Hospitals Keep Medi...,Moving Against Heart Disease
2,What You Need to Know About the Ebola Outbreak,Living With Cancer: Is It Back?,Spending More and Getting Less for Health Care,Doctors Badmouthing Other Doctors,How exercise helps us sleep better.,Blood Pressure Drugs Tied to Risk of Falls,Food and the Dying Patient,Second Drug Is Allowed for Treatment of Ebola,Dental Care in Nursing Homes (1 Letter),Heart Disease and Dogs
3,What you need to know about the Ebola outbreak,Breast Cancer Stories,Health Care Pricing (1 Letter),"For New Doctors, 8 Minutes Per Patient",Ask Well: Sleep or Exercise?,New Concerns About an Old Heart Drug,Fewer Hospitalizations for C.O.P.D. Patients,Second drug is allowed for treatment of Ebola,A Wedding in Intensive Care,Study Questions Fat and Heart Disease Link
4,What you need to know about the Ebola outbreak,Have breast cancer? Tell us your story.,Health Care Act Still Covers 7.3 Million,When Doctors Share With Their Patients,How Exercise Can Help Us Sleep Better,Blood Pressure Drugs May Raise Fall Risk,Smaller military hospitals are said to put pat...,Best Interests of the Patient (1 Letter),What to Look for in Judging the Affordable Ca...,What Running Can Do for the Heart
5,What you need to know about the Ebola outbreak,When Men Get Breast Cancer,How well do you do know health news? Test your...,"In 'Doctored,' Shortcomings of Health Care an...",How Exercise Can Help Us Eat Less,Risks: Stress Tied to Heart Disease and Diabetes,Smaller Military Hospitals Said to Put Patien...,Drugs for Mental Health (1 Letter),A New Way to Pay for Long-Term Care,Cholesterol rates have stopped dropping. Are s...
6,What you need to know about the Ebola outbreak,Faces of Breast Cancer,"Health Care Law Is Not One-Size-Fits-All, and...",When Doctors ‘Google’ Their Patients,"More reasons to exercise for your brain, from ...",Second drug is allowed for treatment of Ebola,The Hospital-Dependent Patient,Drug Could Protect Fertility in Breast Cancer ...,Futile Care at Life’s End,A study questions the link between heart disea...
7,Second drug is allowed for treatment of Ebola,Faces of Breast Cancer,When a Co-Pay Gets in the Way of Health,"Lots of New Patients, Too Few Doctors",Feeling down? Go exercise. Now.,Second Drug Is Allowed for Treatment of Ebola,The invisible patient: the one who's taking c...,Wars on Drugs,A Gap in the Affordable Care Act,People with heart disease may benefit from whi...
8,Second Drug Is Allowed for Treatment of Ebola,Faces of Breast Cancer,California regulates home health care:,A Doctor in the Family,Can Exercise Cause A.L.S.?,This Is Your Brain on Drugs,Patients With Disabilities (1 Letter),Coalition Urges Nations to Decriminalize Drugs...,New Scrutiny for the Affordable Care Act,Obese Children Show Early Signs of Heart Disease
9,"After Ebola Case in Dallas, Health Officials S...","When is cancer actually ""cancer?""",Obama Says Health Care Law Is ‘Working Fine’,When Doctors Give Patients Money,How Exercise Can Help Us Learn,European Regulator Finds Little Risk in Diabet...,Practicing on Patients,Why we have no new psychiatric drugs.,Life Goes On at Long-Term Acute Care Hospitals,Health reporter @ginakolata is answering your ...


## Requirement 9

Concept 1 - Ebola

Concept 2 - Cancer including Breast Cancer

Concept 3 - Health Care Costs- laws and insurance associated with it

Concept 4 - Doctors

Concept 5 - Exercise

Concept 6 - Drugs and risks asociated with them

Concept 7 - Patients

Concept 8 - Drugs for treatment of various diseases

Concept 9 - Affordable Care

Concept 10 - Heart Disease

The above definitions are arrived at by looking at the data from top 10 terms and top 10 tweets. We do have some similar concepts like drugs and healthcare costs. For concept 3, the top 5 terms have law, insurance along with healthcare. So it has more to do with the laws around healthcare and health insurance. Concept 9 on the opther hand talks more about the affordability of healthcare. 

Concept  6 and 8 have the same term drug as its top term. But concept 6 seems to talk more about the risks associated with dtugs which is also the 2nd most frequent term. Concept 8 on the other hand talks about the treatment of various diseases.

Other concepts are clear to define by correlating the top 10 terms and top 10 concepts under it.